# Performance e Operações Paralelas

In [56]:
using Pkg

using BenchmarkTools
using LoopVectorization
using Plots
using StaticArrays
using ThreadsX

using CUDA # há vá?!?!

using Random: seed!, shuffle
using Statistics: mean

# Random Seed
seed!(123)

# Printar cores no terminal
using ANSIColoredPrinters

# Checklist basico para _performance_

1. Arrumar instabilidade de tipo
2. Usar variáveis locais ao invés de gloabais
3. Deixar tudo imutável se possível
4. Desativar checagem de índice em operações com `Array`
5. Ativar suporte SIMD em todos os loops `for` (Single Instruction Multiple Data)

# 1. Arrumar instabilidade de tipo

> Tipo de saída de uma função é __imprevisível__ a partir dos tipos de entradas. Em particular, isso significa que o tipo de saída __pode variar__ dependendo dos valores das entradas.

In [2]:
function positivo(x)
    if x > 0
        return x
    else
        return 0
    end
end

positivo (generic function with 1 method)

> função com untabilidade de tipo >>> `x` é o que?

`@code_warntype` avalia a função como um argumento e prenta um _Abstract Syntax Tree_ (AST)

In [3]:
@code_warntype positivo(-3.4)

Variables
  #self#::Core.Const(positivo)
  x::Float64

Body::Union{Float64, Int64}
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0


> onde ficou vermelho, tem algum problema de tipo

### Arrumando a instabilidade de tipo

Anotar os tipos!

In [4]:
function positivo_stabel(x::AbstractFloat)
    if x > 0
        return x 
    else
        return 0.0
    end
end

positivo_stabel (generic function with 1 method)

In [5]:
function positivo_stabel(x::Integer)
    if x > 0
        return x 
    else
        return 0
    end
end

positivo_stabel (generic function with 2 methods)

In [6]:
@code_warntype positivo_stabel(-3.4)

Variables
  #self#::Core.Const(positivo_stabel)
  x::Float64

Body::Float64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0.0


In [7]:
@code_warntype positivo_stabel(-3)

Variables
  #self#::Core.Const(positivo_stabel)
  x::Int64

Body::Int64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0


### Pq é importante anotar os tipos?

In [8]:
x = rand(1_000);

In [9]:
@benchmark positivo.($x)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.960 μs …  1.343 ms  ┊ GC (min … max):  0.00% … 99.38%
 Time  (median):     3.830 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.102 μs ± 36.201 μs  ┊ GC (mean ± σ):  18.09% ±  3.58%

  ▅▂    ██▅▃▃▂▂▁▂▃▃▂▂▁        ▄▆▅▄▃▂▂▁▁                      ▂
  ██▇▇▇▇████████████████▇▆▅▆▆▇█████████▇███▆▇▇▇▇███████▇▇▇▆▇ █
  1.96 μs      Histogram: log(frequency) by time     15.5 μs <

 Memory estimate: 8.22 KiB, allocs estimate: 7.

In [10]:
@benchmark positivo_stabel.($x)

BenchmarkTools.Trial: 10000 samples with 174 evaluations.
 Range (min … max):  801.149 ns … 84.583 μs  ┊ GC (min … max):  0.00% … 96.27%
 Time  (median):       1.178 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.050 μs ±  3.333 μs  ┊ GC (mean ± σ):  18.37% ± 12.55%

  ▄█▅▃▂     ▃▃▂▂▁▁                                             ▁
  █████▇█▇█▇███████▇▇▆▅▅▄▄▄▃▄▁▁▄▁▃▃▅▄▁▁▁▁▁▁▁▁▁▁▃▄▅▅▅▆▆▆▆▅▇▆▆▆▆ █
  801 ns        Histogram: log(frequency) by time        17 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

### Dicas!

Dar preferência para usar __tipos abstrtatos__ ao invés de tipos concretos.

`AbstractFlosa` >>> `Float64` or `Float32`  
`Integer` >>> `Int128` or `Int64` or `Int342`

### Tipos paramétricos

São introduzidos com as chaves `{}` e usando palavras-chave `where`

In [11]:
# por exemplo qualquer subtipo de `Real`
subtypes(Real)

8-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 FixedPointNumbers.FixedPoint
 Integer
 Rational
 StatsBase.PValue
 StatsBase.TestStat
 VectorizationBase.AbstractSIMD

In [12]:
function positivo_stable2(x::T) where T <: Real
    if x > 0
        return x
    else
        return 0::T
    end
end

positivo_stable2 (generic function with 1 method)

In [13]:
@code_warntype positivo_stable2(-3.4)

Variables
  #self#::Core.Const(positivo_stable2)
  x::Float64

Body::Float64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      Core.typeassert(0, $(Expr(:static_parameter, 1)))
└──      Core.Const(:(return %4))


In [14]:
@code_warntype positivo_stable2(-3)

Variables
  #self#::Core.Const(positivo_stable2)
  x::Int64

Body::Int64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─ %4 = Core.typeassert(0, $(Expr(:static_parameter, 1)))::Core.Const(0)
└──      return %4


### Funciona tambem com `Array`s

* `AbstractArray{T, N}` 
* `AbstractMatrix{T}` atalho para `AbstractArray{T, 2}` 
* `AbstractVector{T}` atalho para `AbstractArray{T, 1}` 

In [15]:
# específico para vector
function meus_zeros(X::AbstractVector{T}) where T <: Real
    return zeros(eltype(x), size(x))    
end

meus_zeros (generic function with 1 method)

In [16]:
# Generalizado para qualquer dimenmção de Array
function meus_zeros(X::AbstractArray{T, N}) where T <: Real where N <: Integer
    return zeros(eltype(x), size(x))    
end

meus_zeros (generic function with 2 methods)

In [17]:
@code_warntype meus_zeros([1, 0, 3])

Variables
  #self#::Core.Const(meus_zeros)
  X::Vector{Int64}

Body::Any
1 ─ %1 = Main.eltype(Main.x)::Any
│   %2 = Main.size(Main.x)::Any
│   %3 = Main.zeros(%1, %2)::Any
└──      return %3


# 2. Usar variáveis locais ao invés de gloabais

Com variáveis globais o compilador LLVM tem dificuldades em otimizar o código Assembly

In [18]:
# var global 
x = rand(1_000);

In [19]:
function sum_global()
    s = 0.0
    for i ∈ x 
        s += i
    end
    return s
end

sum_global (generic function with 1 method)

In [20]:
@benchmark sum_global()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   78.700 μs …   8.218 ms  ┊ GC (min … max): 0.00% … 98.30%
 Time  (median):      80.200 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   107.664 μs ± 204.285 μs  ┊ GC (mean ± σ):  5.60% ±  3.24%

  █▄▂▁▂▂  ▂        ▃▂▁                                          ▁
  ██████████▇▆▇▇█▇███████████▇▆▆▆▆▆▇▆▅▆▅▅▅▅▄▄▅▅▄▃▅▃▄▅▃▃▅▄▃▄▄▃▅▄ █
  78.7 μs       Histogram: log(frequency) by time        314 μs <

 Memory estimate: 70.16 KiB, allocs estimate: 3490.

In [21]:
function sum_arg(x)
    s = 0.0
    for i ∈ x 
        s += i
    end
    return s
end

sum_arg (generic function with 1 method)

In [22]:
@benchmark sum_arg($x)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.100 μs …  4.370 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.110 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.116 μs ± 74.873 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄  █  ▄                                                    ▁
  █▁▁█▁▁█▁▁▇▁▁▇▁▁█▁▁▇▁▁▅▁▁▄▁▁▅▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁█ █
  1.1 μs       Histogram: log(frequency) by time     1.29 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [23]:
@code_warntype sum_global()

Variables
  #self#::Core.Const(sum_global)
  @_2::Any
  s::Any
  i::Any

Body::Any
1 ─       (s = 0.0)
│   %2  = Main.x::Any
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_2::Any
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Any
│         (s = s + i)
│         (@_2 = Base.iterate(%2, %9))
│   %12 = (@_2 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return s


In [24]:
@code_warntype sum_arg(x)

Variables
  #self#::Core.Const(sum_arg)
  x::Vector{Float64}
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  i::Float64

Body::Float64
1 ─       (s = 0.0)
│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}::Tuple{Float64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (s = s + i)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return s


## Se tiver de usar variáveis globeis, use `const`

In [25]:
# var global constante
const const_x = rand(1_000);

In [26]:
function sum_const_global()
    s = 0.0
    for i ∈ const_x 
        s += i
    end
    return s
end

sum_const_global (generic function with 1 method)

In [27]:
@benchmark sum_const_global()

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.100 μs …  61.090 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.110 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.154 μs ± 715.223 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █                                                           
  ▃█▅▁▂▄▁▄▃▁▂▂▂▁▂▂▁▂▂▁▂▂▁▂▂▂▁▂▂▁▂▂▁▂▂▂▁▂▂▁▂▂▁▂▂▁▂▂▂▁▂▂▁▂▂▁▂▂▂ ▂
  1.1 μs          Histogram: frequency by time        1.51 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [28]:
@code_warntype sum_const_global()

Variables
  #self#::Core.Const(sum_const_global)
  @_2::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  i::Float64

Body::Float64
1 ─       (s = 0.0)
│   %2  = Main.const_x::Core.Const([0.4125969203275026, 0.5551098615656092, 0.240819002168303, 0.3965751905591446, 0.8785777477484837, 0.8559115150020329, 0.5722891892469402, 0.955608370221668, 0.5821652710126011, 0.30942078587307686, 0.3247860797558668, 0.24900774646163404, 0.265146177131381, 0.048977939909959245, 0.4467992969582193, 0.08283564682071676, 0.04944164473060653, 0.07956302223576484, 0.20789828761025375, 0.1294812292053089, 0.716321412986181, 0.8443808914960056, 0.9514839964844957, 0.8423388789716104, 0.49510437280583464, 0.5253371662810817, 0.27701498572864747, 0.37308104673931464, 0.1727720764019487, 0.2939256224532212, 0.004889915718355287, 0.7410506462885127, 0.12499304478405016, 0.495218792191364, 0.5677843328878067, 0.6318138413671415, 0.500002367442302, 0.7028780975628444, 0.5266319317099937, 0.865391429380922, 0.

# 3. Deixar tudo imutável se possível

Tudo que é mutável faz com que o compilador LLVM não saiv=ba o que vem pela frente e não consiga otimizar.

## Tuplas vs Arrays

Tuplas são __imutáveis__ e Arrays podem ser modificados após a instanciação.

In [29]:
rand_tuple_point() = (rand(), rand())

rand_tuple_point (generic function with 1 method)

In [33]:
rand_vector_point() = [rand(), rand()]

rand_vector_point (generic function with 1 method)

In [34]:
tuple_points = [rand_tuple_point() for _ ∈ 1:500];

In [35]:
vector_points = [rand_vector_point() for _ ∈ 1:500];

In [36]:
function difference_matrix(points)
    return [p1 .- p2 for p1 in points, p2 in points]
end

difference_matrix (generic function with 1 method)

In [37]:
@benchmark difference_matrix($tuple_points)

BenchmarkTools.Trial: 2404 samples with 1 evaluation.
 Range (min … max):  809.000 μs … 25.752 ms  ┊ GC (min … max):  0.00% … 91.76%
 Time  (median):       1.294 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.050 ms ±  2.362 ms  ┊ GC (mean ± σ):  24.96% ± 19.73%

  ▆█▇▆▅▃▃▂▂▂ ▁▁                                                ▁
  █████████████▇█▄▆▇▇▆▆▅▄▅▄▄▁▄▁▄▁▅█▄▃▅▃▄▆▆▆▆▄▅▄▆▆▆▃▆▇▄▆▃▃▅▅▄▄▃ █
  809 μs        Histogram: log(frequency) by time      13.1 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 2.

In [39]:
@benchmark difference_matrix($vector_points)

BenchmarkTools.Trial: 189 samples with 1 evaluation.
 Range (min … max):  10.654 ms … 63.310 ms  ┊ GC (min … max):  0.00% … 24.82%
 Time  (median):     25.094 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   26.514 ms ± 11.566 ms  ┊ GC (mean ± σ):  16.85% ± 18.69%

     █ ▂   ▂  ▂    ▃         ▂ ▂                               
  ██▆█▇█▇▆▅█▇▆█▃█▃▆█▇▆▅█▅▇▆█▃█▆█▇▃▅▃▅▄▄▃▄▄▄▃▁▄▃▃▁▁▁▃▃▁▃▃▁▃▁▃▃ ▃
  10.7 ms         Histogram: frequency by time        58.7 ms <

 Memory estimate: 24.80 MiB, allocs estimate: 250002.

## `struct` vs `mutable struct`

Jóia        Imutável    
Jóia^-1     Mutável

In [46]:
abstract type Point end

In [47]:
struct ImmutablePoint <: Point 
    x::Float64
    y::Float64
end

In [48]:
mutable struct MutablePoint <: Point 
    x::Float64
    y::Float64
end

In [49]:
function mean_point(p::Point)
    return mean([p.x, p.y])
end

mean_point (generic function with 1 method)

In [50]:
@benchmark mean_point.([ImmutablePoint(rand(), rand()) for _ ∈ 1:500_000])

BenchmarkTools.Trial: 107 samples with 1 evaluation.
 Range (min … max):  33.640 ms … 67.467 ms  ┊ GC (min … max):  0.00% … 13.49%
 Time  (median):     45.714 ms              ┊ GC (median):    16.02%
 Time  (mean ± σ):   47.017 ms ±  7.826 ms  ┊ GC (mean ± σ):  15.27% ±  7.69%

                 ▁ █ ▂  ▅                                      
  ▃█▃▅▅▃▁▁▃▃▁█▆█▁█▆█▅█▆██▆▆▅▆▅▁█▁▃▁▃▃▅▃▆▁▆▁▁▁▅▁▅▁▁▅▃▁▃▁▃▁▁▁▅▃ ▃
  33.6 ms         Histogram: frequency by time        67.1 ms <

 Memory estimate: 57.22 MiB, allocs estimate: 500004.

In [51]:
@benchmark mean_point.([MutablePoint(rand(), rand()) for _ ∈ 1:500_000])

BenchmarkTools.Trial: 84 samples with 1 evaluation.
 Range (min … max):  35.122 ms … 122.437 ms  ┊ GC (min … max):  0.00% … 17.11%
 Time  (median):     56.933 ms               ┊ GC (median):    30.98%
 Time  (mean ± σ):   59.697 ms ±  12.231 ms  ┊ GC (mean ± σ):  28.69% ±  7.48%

                ▅   █   ▅                                       
  ▅▁▁▃▁▁▃▁▁▁▁▁▅▃█▇▆██▇▅▅█▆▅▅▇▆▃▃▆▃▆▁▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▃ ▁
  35.1 ms         Histogram: frequency by time          102 ms <

 Memory estimate: 68.66 MiB, allocs estimate: 1000004.

## `StaticArrays.jl`

__Arrays estáticas em Julia__

* `StaticArray{Size, T, Dims} <: AbstractArray{T, Dims}`    
* `sVector{N, T}`: alias para `StaticArray{N, T, 1}` 
* `sMatrix{M, N, T}`: alias para `StaticArray{(M, N), T, 2}`    

<br>
<br>

__Beenchmarks__ para matrizes `Float64` 3x3:


|__Operação__|*__Speedup__*|
|------------|:-----------:|
|multiplicação|5.9x|
|adição|33.1x|
|determinante|112.9x|
|inversa|67.8x|
|decomposição de autovetores|25.0x|
|decomposição Cholesky|8.8x|
|decomposição LU|6.1x|
|decomposição QR|65.0x|

<br>
<br>
<br>
        
> __Quando usar?__ Como regra-geral, se vc tiver uma `Array` de __até 100 elementos__ é interessante usar uma `StaticArray`.

### Instanciamento

In [52]:
abstract type MeuTipo end

In [53]:
struct MyImmutable <: MeuTipo
    x::Vector{Int}
end

In [54]:
mutable struct MyMutable <: MeuTipo
    x::Vector{Int}
end

In [58]:
struct MySArray <: MeuTipo
	x::SVector{2, Int}
end

In [59]:
function f_immutable()
    for i ∈ 1:1_000
        x = MyImmutable([rand(Int), rand(Int)])
    end
    return nothing
end

f_immutable (generic function with 1 method)

In [60]:
@benchmark f_immutable()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  34.000 μs …  14.304 ms  ┊ GC (min … max):  0.00% … 99.42%
 Time  (median):     42.600 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   72.199 μs ± 344.177 μs  ┊ GC (mean ± σ):  16.33% ±  3.70%

  █▇▅▅▅▄▄▄▃▂▁▂▁▁▁                                              ▂
  █████████████████▇▆▇▇▅▇▆▆▅▅▄▆▅▅▆▅▄▅▆▅▅▅▃▄▅▄▅▃▄▁▃▅▃▄▄▁▃▄▄▃▁▁▄ █
  34 μs         Histogram: log(frequency) by time       376 μs <

 Memory estimate: 93.75 KiB, allocs estimate: 1000.

In [61]:
function f_mutable()
    for i ∈ 1:1_000
        x = MyImmutable([rand(Int), rand(Int)])
    end
    return nothing
end

f_mutable (generic function with 1 method)

In [62]:
@benchmark f_mutable()

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  33.700 μs …  12.811 ms  ┊ GC (min … max):  0.00% … 99.55%
 Time  (median):     39.100 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   62.440 μs ± 320.505 μs  ┊ GC (mean ± σ):  18.39% ±  3.71%

  ██▆▆▄▄▃▄▄▃▃▃▂▁▂▂▃▁▁                                          ▂
  █████████████████████▇▆▇▅▆▇▆▇▇▆▆▆▆▄▆▄▅▆▄▄▄▅▆▃▄▄▄▃▄▄▃▄▅▂▅▄▃▄▅ █
  33.7 μs       Histogram: log(frequency) by time       194 μs <

 Memory estimate: 93.75 KiB, allocs estimate: 1000.

In [63]:
function f_sarray()
    for i ∈ 1:1_000
        x = MySArray(SVector(rand(Int), rand(Int)))
    end
    return nothing
end

f_sarray (generic function with 1 method)

In [64]:
@benchmark f_sarray()

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):  7.125 μs … 241.425 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.575 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.458 μs ±   4.772 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁█▇▁ ▃▂         ▁                                           ▁
  ████████▇▇▆▇▇████████▇█▇▇█▇▇▆▅▅▄▄▅▄▆▅▅▅▅▆▄▂▃▃▄▃▄▃▃▃▄▃▃▃▄▂▃▃ █
  7.12 μs      Histogram: log(frequency) by time      21.4 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

### Operações

In [65]:
function mean_meu_tipo(m::MeuTipo)
    return mean(m.x)
end

mean_meu_tipo (generic function with 1 method)

In [66]:
@benchmark mean_meu_tipo(MyImmutable([rand(Int), rand(Int)]))

BenchmarkTools.Trial: 10000 samples with 991 evaluations.
 Range (min … max):  45.510 ns …  11.848 μs  ┊ GC (min … max):  0.00% … 99.29%
 Time  (median):     49.647 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   77.758 ns ± 310.580 ns  ┊ GC (mean ± σ):  14.33% ±  3.69%

  █▆▄▃▁▃▄▃▂▂▁▂▂▁▁ ▁                                            ▁
  ███████████████████▇▇▇▇▆▆▆▅▅▄▅▅▅▅▄▅▅▅▅▃▄▄▅▄▄▅▅▄▄▄▃▄▃▄▄▄▃▃▄▃▄ █
  45.5 ns       Histogram: log(frequency) by time       276 ns <

 Memory estimate: 96 bytes, allocs estimate: 1.

In [67]:
@benchmark mean_meu_tipo(MyMutable([rand(Int), rand(Int)]))

BenchmarkTools.Trial: 10000 samples with 991 evaluations.
 Range (min … max):  45.005 ns …   8.180 μs  ┊ GC (min … max):  0.00% … 98.79%
 Time  (median):     49.041 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   70.854 ns ± 259.556 ns  ┊ GC (mean ± σ):  13.24% ±  3.69%

  ▆█▅▄▃▂▁▁▄▄▂▂▁▁ ▁▃▂▁▁                                         ▂
  ██████████████▇███████▇█▇▆▇▇▇▇▇▆▆▆▆▅▆▅▅▅▅▅▅▆▄▅▄▄▄▄▁▄▄▁▄▄▅▄▁▃ █
  45 ns         Histogram: log(frequency) by time       210 ns <

 Memory estimate: 96 bytes, allocs estimate: 1.

In [68]:
@benchmark mean_meu_tipo(MySArray(SVector(rand(Int), rand(Int))))

BenchmarkTools.Trial: 10000 samples with 999 evaluations.
 Range (min … max):  13.413 ns …  1.730 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.815 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   21.996 ns ± 25.554 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █▆ ▂       ▄▄ ▁▂▂ ▁▁        ▂▃▅▃▁                          ▁
  ▇██▆██▇█▇▆▆▅███████████▇██████████▆▆▇▇█▅▄▄▅▄▄▄▄▄▃▄▄▃▄▃▄▃▄▃▄ █
  13.4 ns      Histogram: log(frequency) by time      53.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# 4. Desativar checagem de índice em operações com `Array` - `@inbounds`

A maioria das ligaugens modernas, poe questões de segurança, garantem que não haverá acesso fora dos elementos de uma array.    
<br>
Porém, em algumas situações que _performance_ é crítica, você pode remover essa checagem de acesso. 
<br>
Para fazer isso em Juia é só usar o macro `@inbounds`
<br>
>___Cuidado!!!___ Remover o _bound check_ de Julia é __perigoso__. Se certifique que não está fazendo um loop inseguro antes de usar `@inbounds`

In [73]:
array_x = rand(10_000);

In [74]:
array_y = rand(10_000);

In [72]:
function inner(x, y)
    s = zero(eltype(x))
    for i ∈ eachindex(x)
        s += x[i]*y[i]
    end
    return s
end

inner (generic function with 1 method)

In [77]:
@benchmark inner($array_x, $array_y)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  11.400 μs … 450.800 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.500 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.363 μs ±  10.201 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄▃▁▂▃▂▂▁                                                    ▁
  ███████████▇▅▆▇▆▆▄▅▅▅▅▅▅▅▄▄▄▄▄▃▃▁▄▁▃▃▄▁▃▄▃▁▁▃▁▄▄▁▄▁▃▁▃▁▃▄▅▄▅ █
  11.4 μs       Histogram: log(frequency) by time        28 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [78]:
function inner_inbound(x, y)
    s = zero(eltype(x))
    for i ∈ eachindex(x)
        @inbounds s += x[i]*y[i]
    end
    return s
end

inner_inbound (generic function with 1 method)

In [79]:
@benchmark inner_inbound($array_x, $array_y)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  11.400 μs … 114.500 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.500 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.113 μs ±   3.944 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄▃▁▁▃▂▂▁ ▁                                                  ▁
  ███████████▇▅▆▆▆▇▅▄▄▄▅▅▄▃▃▅▄▃▄▄▄▅▃▁▄▁▃▁▁▃▄▃▄▃▄▄▃▁▄▃▁▁▁▃▃▅▄▄▄ █
  11.4 μs       Histogram: log(frequency) by time      27.5 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

# 5. Ativar suporte SIMD em todos os loops `for` (Single Instruction Multiple Data)